# Sending and Receiving Packets

In [ ]:
from scapy.all import *

In [ ]:
conf.verb=1
conf.color_theme = RastaTheme()

## Stacking Layers
The / operator has been used as a composition operator between two layers. When doing so, the lower layer can have one or more of its defaults fields overloaded according to the upper layer. (You still can give the value you want). A string can be used as a raw layer.
<img src="http://scapy.readthedocs.io/en/latest/_images/fieldsmanagement.png" />

In [ ]:
IP()

In [ ]:
IP()/TCP()

In [ ]:
Ether()/IP()/TCP()

In [ ]:
IP()/TCP()/"GET / HTTP/1.0\r\n\r\n"

In [ ]:
Ether()/IP()/IP()/UDP()

In [ ]:
IP(proto=55)/TCP(dport=22)

## Build ARP packets
- build ARP packet which has two layers - Ether and ARP

In [ ]:
ls(Ether)

In [ ]:
ls(ARP)

In [ ]:
arpPkt = Ether()/ARP()

In [ ]:
arpPkt.show()

In [ ]:
# Change Ethernet protocol's destination field to broadcase MAC address
arpPkt['Ethernet'].dst= 'FF:FF:FF:FF:FF:FF'

In [ ]:
arpPkt.show()

In [ ]:
# change ARP protocol's hwdst to broadcast MAC address
#arpPkt['Ethernet']['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'
arpPkt['ARP'].hwdst = 'FF:FF:FF:FF:FF:FF'

In [ ]:
arpPkt.show()

## Capture packet using wireshark
- ARP packet is ready to be sent
- Open Wireshark and start capturing traffic

# Send packets - one way
- sendp() - works at layer two; can send ARP, Ethernet, etc.
- send() - send packets at layer 3 (IP, TCP, HTTP, etc.); will handle routing and layer 2
    - if return_packets=True is passed as parameter, these functions will also return received packes

In [ ]:
help(send)

In [ ]:
help(sendp)

In [ ]:
# send the arpPkt - use sendp() because ARP is layer 2 packet
sendp(arpPkt)

In [ ]:
# one liner
sendp(Ether(dst="ff:ff:ff:ff:ff:ff",src="00:11:22:aa:bb:cc")/ARP(hwsrc="00:11:22:aa:bb:cc",pdst="192.168.231.2"))

## Sending HTTP request

In [ ]:
httpPkt = IP(dst="www.coloradomesa.edu")/TCP(dport=80)/"GET /index.html HTTP/1.1\r\n\r\n"

In [ ]:
bytes(httpPkt)

In [ ]:
hexdump(httpPkt)

In [ ]:
httpPkt.show()

## Send and Receive Packets
- Send and receive functions are the heart of Scapy
- sr() - send L3 packets and receive answers; returns answered and unanswered packets
- sr1() - returns only one packet that answered the sent packet
- sr() and sr1() functions can only send layer 3 L3 packets (IP, HTTP, etc.)
- srp() - send and receive L2 packets (ARP, Ehternet, 802.3, etc)
- srp1() - send and receive one layer 2 packet

In [ ]:
response = sr(httpPkt)

In [ ]:
response

In [ ]:
response[0].show()

In [ ]:
response[0].summary()

## TCP/IP full three-way handshake

### send SYN packet

In [ ]:
# Create syn packet
syn = IP(dst='www.sustainablesites.org')/TCP(dport=80, flags='S')

In [ ]:
syn

In [ ]:
# Receive SYN-ACK packet from google.com
syn_ack = sr1(syn)

In [ ]:
syn_ack

In [ ]:
# Send Get request
getStr = 'GET / HTTP1.1\r\nHost:www.sustainablesites.org\r\n\r\n'
request = IP(dst='www.sustainablesites.org') / TCP(dport=80, sport=syn_ack[TCP].dport, 
                                        seq=syn_ack[TCP].ack, ack=syn_ack[TCP].seq+1, 
                                        flags='A') / getStr
reply = sr1(request)

In [ ]:
reply

In [ ]:
reply.summary()

In [ ]:
reply.show()

In [ ]:
tcpip = IP(dst="dstIP") / TCP(dport=22) # tcp packet needs dest port
reply = sr(tcpip)
reply

In [ ]:
ans, unans = _ 
ans.summary()

## Create ping request
- IP packet with ICMP echo request and some payload
- send the packet and capture it with wireshark

In [ ]:
ping = IP(dst="domain", src="IP") / ICMP() / "Hey there, are you up?"

In [ ]:
ping.show()

In [ ]:
send(ping)

In [ ]:
reply = sr1(ping)
reply.show()

## Exercise
- Spoof src IP and ping some IP in research/controlled network
- 